In [4]:
### Benjamin Tollison ###
import sympy as sp
from numba import jit
from IPython.display import display, Math, Latex
from sympy.solvers.solveset import linsolve
from sympy.solvers.pde import pdsolve
from sympy import  lambdify, Matrix,sin,cos, sinh, cosh, exp, pi, symbols, integrate, Function, Eq
import numpy as np
import scipy
import matplotlib.pyplot as plt
def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)

In [5]:
rho = 1.2250554566 # kg/m
alpha,delta_E,theta = symbols('\\alpha \\delta_E \\theta')
gamma = theta - alpha
W = 20500*0.453592 # kg
S_W = 300*0.3048**2 # m^2
c_W = 11.32*0.3048 # m
v = 500 * 0.3048 # m/s
q_infty = 0.5*rho*v**2
C_L = 0.14 + 4.2*alpha - 4.8*alpha**2 + 0.44*delta_E
C_M = -0.02 + 0.04*alpha - 0.6*delta_E - 0.08*alpha*delta_E + .08*delta_E**2 + 0.5*alpha**2*delta_E
alpha_0_symbol,delta_E0_symbol = symbols('\\alpha_0 \\delta_{E_0}')
C_L_alpha = lambdify((alpha,delta_E),C_L.diff(alpha))
C_L_delta_E = lambdify((alpha,delta_E),C_L.diff(delta_E))
C_M_alpha = lambdify((alpha,delta_E),C_M.diff(alpha))
C_M_delta_E = lambdify((alpha,delta_E),C_M.diff(delta_E))
C_L_lambda = lambdify((alpha,delta_E),C_L)
C_M_lambda = lambdify((alpha,delta_E),C_M)
alpha_0,delta_E0 = 20,20
iterations = 0
lift_trim = W/(q_infty*S_W)
while True:
  iterations += 1
  if -1e-3 < C_L_lambda(alpha_0,delta_E0) - lift_trim < 1e-3 and -1e-3 < C_M_lambda(alpha_0,delta_E0) < 1e-3:
    break
  elif iterations > 1e7:
    break
  else:
    iterations += 1
    jacobian = np.array([[C_L_alpha(alpha_0,delta_E0),C_L_delta_E(alpha_0,delta_E0)],\
      [C_M_alpha(alpha_0,delta_E0),C_M_delta_E(alpha_0,delta_E0)]])
    b = np.array([C_L_lambda(alpha_0,delta_E0),C_M_lambda(alpha_0,delta_E0)])
    x = np.linalg.inv(jacobian) @ b
    alpha_0 = x[0]
    delta_E0 = x[1]
displayEquations(alpha_0_symbol,alpha_0*180/np.pi)
displayEquations(delta_E0_symbol,delta_E0*180/np.pi)
print('C_L should be really close to {}'.format(lift_trim))
print('C_L = {}'.format(C_L_lambda(alpha_0,delta_E0)))
print('C_M = {}'.format(C_M_lambda(alpha_0,delta_E0)))
print('number of iterations = {}'.format(iterations))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

C_L should be really close to 0.023451594698306277
C_L = 0.02250326889225765
C_M = -0.0008526988393041737
number of iterations = 734317


In [ ]:
# linear approximation
A = np.array([[4.2,0.44],[0.04,-0.6]])
A1 = np.array([[lift_trim-0.14,0.44],[0.02,-0.6]])
A2 = np.array([[4.2,lift_trim-0.14],[0.04,0.02]])
alpha_0 = np.linalg.det(A1) / np.linalg.det(A)
delta_E0 = np.linalg.det(A2) / np.linalg.det(A)
displayEquations(alpha_0_symbol,alpha_0*180/np.pi)
displayEquations(delta_E0_symbol,delta_E0*180/np.pi)
print('C_L should be really close to {}'.format(lift_trim))
print('C_L = {}'.format(C_L_lambda(alpha_0,delta_E0)))
print('C_M = {}'.format(C_M_lambda(alpha_0,delta_E0)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

C_L should be really close to 0.023451594698306277
C_L = 0.020666178465464702
C_M = 2.0189680074379074e-05
